In [1]:
import cv2
import numpy as np
import os

# Initialize video capture object
cap = cv2.VideoCapture(0)

# Load the face cascade classifier
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip = 0
dataset_path = './data/'
face_data = []
labels = []
class_id = 0
names = {}  # Dictionary to map label id to person name

# Data preparation - Load face data from .npy files
for fx in os.listdir(dataset_path):
    if fx.endswith(".npy"):
        # Extract the name from the file name (without .npy extension)
        name = fx[:-4]
        names[class_id] = name
        print("Loaded data for:", name)

        # Load the face data from the numpy file
        data_item = np.load(os.path.join(dataset_path, fx))
        face_data.append(data_item)

        # Create labels (e.g., 0 for first person, 1 for second person, etc.)
        target = class_id * np.ones((data_item.shape[0],), dtype=int)
        labels.append(target)

        # Increment the class ID for the next person
        class_id += 1

# Stack the face data and labels into arrays
face_data = np.vstack(face_data)
labels = np.hstack(labels)

print("Data preparation completed. Total classes:", class_id)

# Train a simple KNN classifier
def knn(train, test, k=5):
    distances = []
    
    for i in range(train.shape[0]):
        # Compute the L2 distance
        dist = np.sqrt(np.sum((train[i, :-1] - test) ** 2))
        distances.append((dist, train[i, -1]))
    
    # Sort based on distances
    distances = sorted(distances, key=lambda x: x[0])
    
    # Get the top k distances
    top_k = np.array(distances[:k])
    
    # Return the most common label (majority voting)
    labels, counts = np.unique(top_k[:, 1], return_counts=True)
    return labels[np.argmax(counts)]

# Append labels as the last column to training data
train_data = np.hstack((face_data, labels.reshape(-1, 1)))

# Start video capture for real-time face classification
while True:
    ret, frame = cap.read()
    if not ret:
        continue

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)

    for face in faces:
        x, y, w, h = face
        offset = 10
        face_section = frame[y - offset:y + h + offset, x - offset:x + w + offset]
        face_section = cv2.resize(face_section, (100, 100))
        face_section = face_section.flatten()

        # Predict using KNN
        label = knn(train_data, face_section)
        predicted_name = names[int(label)]

        # Display the name and draw a rectangle around the face
        cv2.putText(frame, predicted_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x - offset, y - offset), (x + w + offset, y + h + offset), (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow("Face Recognition", frame)

    # Break the loop if 'q' is pressed
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break

# Release video capture and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


Loaded data for: pradeep
Loaded data for: sachin
Data preparation completed. Total classes: 2
